In [141]:
import Crypto 
import Crypto.Random
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5 
from Crypto.Hash import SHA256
import hashlib
import binascii
import json
import datetime

In [142]:
class Transaction:
    def __init__(self, sender, recipient, value):
        self.sender = sender
        self.recipient = recipient
        self.value = value
    
    def to_dict(self):
        return ({'sender': self.sender, 'recipient': self.recipient, 'value': self.value})
    
    def add_signature(self, signature_):
        self.signature = signature_
    
    def verify_transaction_signature(self):
        if hasattr(self,'signature'):
            public_key = RSA.importKey(binascii.unhexlify(self.sender))
            verifier = PKCS1_v1_5.new(public_key)
            h = SHA256.new(str(self.to_dict()).encode('utf-8'))
            return verifier.verify(h,binascii.unhexlify(self.signature))
        else:
            return False
    
    def to_json(self):
        return json.dumps(self.__dict__,sort_keys = False)

In [143]:
class Wallet:
    def __init__(self):
        random = Crypto.Random.new().read
        self._private_key = RSA.generate(1024,random)
        self._public_key = self._private_key.publickey()
        
    def sign_transaction(self,transaction: Transaction):
        signer = PKCS1_v1_5.new(self._private_key)
        h = SHA256.new(str(transaction.to_dict()).encode('utf-8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')
    
    @property
    def identity(self):
        pubkey = binascii.hexlify(self._public_key.exportKey(format="DER"))
        return pubkey.decode('ascii')
    @property
    def private(self):
        privkey = binascii.hexlify(self._private_key.exportKey(format="DER"))
        return privkey.decode('ascii')

In [144]:
class Block:
    def __init__(self, index,transactions, timestamp, previous_hash):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.hash = None
        self.nonce = 0
    
    def to_dict(self):
        return ({
            'index':self.index,
            'transactions':self.transactions,
            'timestamp':self.timestamp,
            'previous_hash':self.previous_hash,
            'nonce':self.nonce
        })
    
    def to_json(self):
        return json.dumps(self.__dict__)
    
    def compute_hash(self):
        return hashlib.sha256(str(self.to_dict()).encode()).hexdigest()

In [145]:
class Blockchain:
    difficulty = 4
    
    def __init__(self):
        self.unconfirmed_transactions = []
        self.chain = []
        self.create_genesis_block()
        
    def create_genesis_block(self):
        genesis_block = Block(0,[],datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),"0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block.to_json())
    
    def add_new_transaction(self, transaction: Transaction):
        if transaction.verify_transaction_signature():
            self.unconfirmed_transactions.append(transaction.to_json())
            return True
        else:
            return False
    
    def add_block(self, block ,proof):
        previous_hash = self.last_block['hash']
        
        if previous_hash != block.previous_hash:
            return False
        
        if not self.is_valid_proof(block,proof):
            return False
        
        block.hash = proof
        
        self.chain.append(block.to_json())
        return True
    
    def is_valid_proof(self, block, block_hash):
        return (block_hash.startswith('0' * Blockchain.difficulty) and block_hash == block.compute_hash())
    
    def proof_of_work(self, block):
        block.nonce = 0
        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0' * Blockchain.difficulty):
            block.nonce +=1
            computed_hash = block.compute_hash()
        return computed_hash
    
    def mine(self, myWallet):
        block_reward = Transaction("Block_Reward", myWallet.identity, "5.0").to_json()
        self.unconfirmed_transactions.insert(0,block_reward)
        if not self.unconfirmed_transactions:
            return False
        
        new_block = Block(index=self.last_block['index']+1,transactions=self.unconfirmed_transactions,timestamp=datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),
                        previous_hash=self.last_block['hash'])
        proof = self.proof_of_work(new_block)
        if self.add_block(new_block, proof):
            self.unconfirmed_transactions = []
            return new_block
        else:
            return False
    
    @property
    def last_block(self):
        return json.loads(self.chain[-1])

In [146]:
blockchain = Blockchain()
Peter = Wallet()
Mary = Wallet()

t = Transaction(Peter.identity, Mary.identity, 250)
t.add_signature(Peter.sign_transaction(t))
blockchain.add_new_transaction(t)

blockchain.mine(Peter)
print("First Block has a hash value of: ",blockchain.last_block['hash'])
print("And a nonce of: ",blockchain.last_block['nonce'])
print(blockchain.last_block)
blockchain.mine(Peter)
print("Second Block has a hash value of: ",blockchain.last_block['hash'])
print("And a nonce of: ",blockchain.last_block['nonce'])
print(blockchain.last_block)
print("The above program is generated by Suwandi Ryan Loe (student id: 55724681) on", datetime.datetime.now())

First Block has a hash value of:  000081058a7445019ff246457c6cc73510a07bf5c3bd31f6c900ec6306458578
And a nonce of:  31914
{'index': 1, 'transactions': ['{"sender": "Block_Reward", "recipient": "30819f300d06092a864886f70d010101050003818d0030818902818100a6c0ae6e33d75b7533ea1acdc0184ed2cb5833318d4b9b1c10d05bd1e61e9529aac7febbef59d65ffef6e4f486ec07202284ecc260a40f3347ffe404a701504c00826789f22476b810bd66ba8055003f36c023cb09c1b721d542ca5d174ab26f82849af89d3b47cb5ad60b78252bb22ca2c50812f5be8848bf9f5be28e3ca99f0203010001", "value": "5.0"}', '{"sender": "30819f300d06092a864886f70d010101050003818d0030818902818100a6c0ae6e33d75b7533ea1acdc0184ed2cb5833318d4b9b1c10d05bd1e61e9529aac7febbef59d65ffef6e4f486ec07202284ecc260a40f3347ffe404a701504c00826789f22476b810bd66ba8055003f36c023cb09c1b721d542ca5d174ab26f82849af89d3b47cb5ad60b78252bb22ca2c50812f5be8848bf9f5be28e3ca99f0203010001", "recipient": "30819f300d06092a864886f70d010101050003818d003081890281810089c0cebc879bb05b3d6f9c88f32021decb3ef0fea8f81e986